In [ ]:
import bw2data as bw
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter
import networkx as nx
import pandas as pd
from libs import extract as extract
import libs.print_lci as print_lci
import libs.lci as inventory

### Initialization
Initialize the project, database and the output file

In [ ]:
project_name = input("Enter the name of the project: ")
database_name = input("Enter the name of the database in the project: ")
bw.projects.set_current(project_name)
selected_database = bw.Database(database_name)
output_file_name = database_name + '.graphml' #input databasename is used as as the name for the output network

### Check the life cycle inventory

To print the all the details concerning the activities and corresponding exchanges from the selected database. Only use, *if required*

In [ ]:
print_lci.print_all (selected_database)

In [ ]:
print_lci.print_classified(selected_database) #classifies the activities within the selected database.

## LCI Database

LCI_database is generated in the format of [parameters, activities]
- parameters (nested dict and lists)
```
                            {'Project Parameters': [{ }, { }, { }, ...]
                            'Database Parameters': [{ }, { }, { }, ...]
                            'Activity Parameters': [{ }, { }, { }, ...] }
```
- activities (nested dict and list): 
```                         
                            {activity-1{name: 'test',
                                         amount: 0,
                                         ...........
                                         ...........
                                         exchanges: [ { }, { }, { }, .....]
                                                                    }
                                         ..................},
                            activity-2{....
                                        ...................}
                              .............................}
```

### Required keys for flows
Enter the keys that are necessary for each flow types

In [ ]:
keys_technosphere = ['name', 'database', 'location', 'unit','reference product']
keys_biosphere = ['name', 'database', 'unit', 'categories']
keys_production = ['name', 'database', 'location', 'unit','reference product']

### Generate LCI database as a list of dictionaries

In [ ]:
lci_db = inventory.LCI(project_name, database_name)
LCI_list = lci_db.LCI_database_list(keys_biosphere, keys_technosphere, keys_production)
import pprint
pprint.pprint(LCI_list, indent=2)

## LCI to network diagram
Convert the activities, the products, technosphere and biosphere flows into an interconnected network diagram. This resultant file is stored in the graphml file format. Graphml allows much more flexibility in terms of export and editing using open-source graphml editing programs such as gephi and cytoscape. 

### Node & Attributes
Creates nodes, edges, and stores all the attributes of nodes into a dictionary

In [ ]:
network_graph, node_attributes = inventory.LCI.network(LCI_list)
pprint.pprint(node_attributes)

### Clean-up 

The node attributes dictionary is checked and cleaned if there any key:value pairs other than the numeric or string. This is because 'graphml' only allows datatypes of bool, numbers (float, int etc.) and strings. Any other datatype as node or edge attributes will result in a "type" error. Another option could be use the gml format, i.e a format created by the team behind networkx - [GML format](https://networkx.org/documentation/stable/reference/readwrite/generated/networkx.readwrite.gml.write_gml.html). *Another limitation* of this implementation is that [substitution exchanges](https://docs.brightway.dev/en/latest/content/theory/theory.html) (multi-output processes) are not considered.


In [ ]:
keys_to_remove, datatypes_keys, all_key_values_types = inventory.LCI.lci_key_search(node_attributes)
print ('They keys to remove are: ', keys_to_remove, ' with datatypes ', datatypes_keys, ', respectively', "\n")
pprint.pprint(all_key_values_types)

**Note** The intrinsic advantage of using brightway is that the data within the LCI is not hidden behind any database. It is flexible and can be stored in dictionary in any formats (as per the requirements of a specific project). Therefore, it may not be viable to create a function to convert all the incomaptible datatypes present within any LCI before it can be written to a network. This will be project dependent. Thus, the below code is just an example. It converts the 'cateorgy' tuples present within the biosphere flows to a string 

In [ ]:
def recursive_lci_search(node_details):
        for key, value in node_details.items():
                # print (key)
                if key == 'categories' and isinstance(value, tuple):
                        # print(key, value, '::'.join(value))
                        node_details[key] = '::'.join(value)
                if value is None:
                        node_details[key] = 'none'
                if isinstance(value, dict):
                        recursive_lci_search(value)
        return (node_details)
updated_node_attributes = recursive_lci_search(node_attributes)
pprint.pprint(updated_node_attributes)

#check keys again for datatype conflicts
keys_to_remove, datatypes_keys, all_key_values_types = inventory.LCI.lci_key_search(updated_node_attributes)
print ('They keys to remove are: ', keys_to_remove, ' with datatypes ', datatypes_keys, ', respectively', "\n")

In [ ]:
# Use with caution. This is a nuclear option to drop all the incompatible keys (found above) from the node_attributes dictionary. Data will be LOST!
cleaned_node_attributes = updated_node_attributes
for key in list(updated_node_attributes):
    for key2 in list(updated_node_attributes[key]):
        if key2 in keys_to_remove:
            del updated_node_attributes[key][key2]
pprint.pprint(cleaned_node_attributes)

### Write the network

In [ ]:
#Export to graphml format
nx.set_node_attributes(network_graph, updated_node_attributes)
nx.write_graphml(network_graph, output_file_name, prettyprint=True)